In [ ]:
%pip install crewai crewai_tools pandas groq

In [1]:
from crewai import Agent, Task, Crew, LLM
import os
import yaml 
from dotenv import load_dotenv

In [79]:
from typing import Type
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

class SleepToolInput(BaseModel):
    """Schema for SleepTool arguments."""
    seconds: int = Field(20, description="Number of seconds to sleep")

class SleepTool(BaseTool):
    name: str = "sleep"
    description: str = "Pauses execution for a specified number of seconds."
    args_schema: Type[BaseModel] = SleepToolInput

    def _run(self, seconds: int) -> str:
        import time
        time.sleep(seconds)
        return f"Slept for {seconds} seconds"
sleep_tool = SleepTool()


In [80]:
import os
from dotenv import load_dotenv

load_dotenv(".env", override=True)

PRIMARY_KEY   = os.getenv("GROQ_API_KEY_PRIMARY")
SECONDARY_KEY = os.getenv("GROQ_API_KEY_SECONDARY")
serper_key= os.getenv("SERPER_API_KEY")
os.environ["SERPER_API_KEY"]=serper_key
from crewai import LLM

# LLM #1 uses your “primary” key
llm1 = LLM(
    model="groq/gemma2-9b-it",
    api_key=PRIMARY_KEY,      # overrides the default GROQ_API_KEY
    temperature=0.2,
)

# LLM #2 uses your “secondary” key
llm2 = LLM(
    model="groq/gemma2-9b-it",
    api_key=SECONDARY_KEY,    # separate authentication
    temperature=0.4,
)


In [106]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

In [107]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")


In [108]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool


In [109]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool(),sleep_tool],
  llm=llm1,

)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool(),sleep_tool],
  llm=llm2,

)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool(),sleep_tool],
  llm=llm1,

)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)


In [110]:
# Create the agent
email_content_specialist = Agent(
    config=email_agents_config['email_content_specialist'],
    llm=llm2,
    
)

engagement_strategist = Agent(
    config=email_agents_config['engagement_strategist'],
    llm=llm1

)

# Create the tasks

email_drafting = Task(
    config=email_tasks_config['email_drafting'],
    agent=email_content_specialist
)
engagement_optimization = Task(
    config=email_tasks_config['engagement_optimization'],
    agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
    agents=[
        email_content_specialist,
        engagement_strategist
    ],
    tasks=[
        email_drafting,
        engagement_optimization
    ],
    verbose = True
)

In [114]:
from crewai import Flow
from crewai.flow.flow import listen, start
import pandas as pd

df= pd.DataFrame()

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):

        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores


    @listen(score_leads)
    def store_leads_score(self, scores):

        return scores
    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        print('a')
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()


╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: SalesPipeline                                                                                            │
│  ID: 3f7adca7-a43e-4c44-b74b-d41c3d9cac57                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [115]:
flow.plot()

Plot saved as crewai_flow.html


In [116]:
emails = await flow.kickoff_async()

🌊 Flow: SalesPipeline
    ID: 3f7adca7-a43e-4c44-b74b-d41c3d9cac57
└── 🧠 Starting Flow...

 Flow started with ID: 3f7adca7-a43e-4c44-b74b-d41c3d9cac57


🌊 Flow: SalesPipeline
    ID: 3f7adca7-a43e-4c44-b74b-d41c3d9cac57
├── 🧠 Starting Flow...
└── 🔄 Running: fetch_leads

🌊 Flow: SalesPipeline
    ID: 3f7adca7-a43e-4c44-b74b-d41c3d9cac57
├── Flow Method Step
└── ✅ Completed: fetch_leads

🌊 Flow: SalesPipeline
    ID: 3f7adca7-a43e-4c44-b74b-d41c3d9cac57
├── Flow Method Step
├── ✅ Completed: fetch_leads
└── 🔄 Running: score_leads

...

🌊 Flow: SalesPipeline
    ID: 3f7adca7-a43e-4c44-b74b-d41c3d9cac57
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
└── ✅ Completed: write_email

🌊 Flow: SalesPipeline
    ID: 3f7adca7-a43e-4c44-b74b-d41c3d9cac57
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── 🔄 Running: send_email

🌊 Flow: SalesPipeline
    ID: 3f7adca7-a43e-4c44-b74b-d41c3d9cac57
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── ✅ Completed: send_email

✅ Flow Finished: SalesPipeline
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── ✅ Completed: send_email

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: SalesPipeline                                                                                            │
│  ID: 3f7adca7-a43e-4c44-b74b-d41c3d9cac57                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [120]:
import pandas as pd

# Assume this is your saved output from CrewAI
scores_saved = flow.state["score_crews_results"]

# Create lists to collect info
lead_info = []
company_info = []
score_info = []

for crew_output in scores_saved:
    if crew_output.pydantic:  # Only if pydantic data exists
        personal = crew_output.pydantic.personal_info
        company = crew_output.pydantic.company_info
        score = crew_output.pydantic.lead_score
        
        lead_info.append({
            "name": personal.name,
            "job_title": personal.job_title,
            "role_relevance": personal.role_relevance,
            "professional_background": personal.professional_background
        })
        
        company_info.append({
            "company_name": company.company_name,
            "industry": company.industry,
            "company_size": company.company_size,
            "revenue": company.revenue,
            "market_presence": company.market_presence
        })
        
        score_info.append({
            "score": score.score,
            "scoring_criteria": score.scoring_criteria,
            "validation_notes": score.validation_notes
        })

# Create 3 DataFrames
lead_df = pd.DataFrame(lead_info)
company_df = pd.DataFrame(company_info)
score_df = pd.DataFrame(score_info)


In [125]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.2 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0043


C:\Users\lenovo\AppData\Local\Temp\ipykernel_1416\3696135900.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,21747,17471,0,4276,10


In [126]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0002


C:\Users\lenovo\AppData\Local\Temp\ipykernel_1416\99626748.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,1104,916,0,188,2


In [127]:
scores = flow.state["score_crews_results"]

In [128]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,John Doe
Job Title,Software Engineer
Role Relevance,9
Professional Background,Experienced software engineer with a focus on AI and machine learning.
Company Name,Example Company
Industry,Technology
Company Size,500
Revenue,None
Market Presence,8
Lead Score,75


In [129]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

John, as a Software Engineer at Example Company, you understand the power of AI
and machine learning.  CrewAI's Multi-Agent Orchestration Platform can help you
automate complex tasks and workflows across your technology company. We empower
you to orchestrate AI agents for any vertical, unlocking new levels of
efficiency and innovation.  Ready to see how CrewAI can transform your
workflows?  Schedule a demo today: [Link to scheduling tool]


In [131]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "João Moura",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "joao@clearbit.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads)
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

In [132]:
flow = SalesPipeline()
flow.plot()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: SalesPipeline                                                                                            │
│  ID: 3e1b30f3-fdec-4373-ae5d-ce6bad70701b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Plot saved as crewai_flow.html


In [133]:
from IPython.display import IFrame

IFrame(src='./crewai_flow_complex.html', width='150%', height=600)